<a href="https://colab.research.google.com/github/xHexlabx/SuperAI_SS4_Recap/blob/main/SuperAI_SS4_Level_1/Hack_1_Image_Search/Image_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SuperAI Season 4 - Level 1 Hackathon - Image Search

# Mounting Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Kaggle

In [2]:
!pip install -q kaggle

In [3]:
#kaggle setting for competition
from google.colab import userdata

username = userdata.get('KAGGLE_USER')
key = userdata.get('KAGGLE_KEY')
# Echo the credentials into the kaggle.json file
!mkdir -p ~/.kaggle
!echo '{{"username":"{username}","key":"{key}"}}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

# Explore Datasets

In [4]:
import cv2

In [5]:
def refine_image (image) :

    width = 384
    height = 384

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image , (width , height))

    return image

In [6]:
refined_image_path = '/content/drive/MyDrive/SuperAI_SS4/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/queries/refined_queries'

for i in range(22) :

    image = cv2.imread(f'/content/drive/MyDrive/SuperAI_SS4/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/queries/queries/{i}.jpg')

    resized_image = refine_image(image)
    image = cv2.cvtColor(resized_image, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f'{refined_image_path}/{i}.jpg' , resized_image)

# CLIP VisionModel For Encode Image

In [7]:
!nvidia-smi

Tue May 21 06:46:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   40C    P8              12W /  72W |      1MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPModel
import torch

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CLIPModel.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K")
processor = AutoProcessor.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [10]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [11]:
inputs = processor(images = image, return_tensors = "pt")
image_features = model.get_image_features(**inputs)

In [12]:
image_features.shape

torch.Size([1, 1024])

# Solution-CLIP Zeroshot For Image Similarity Search

## Cosine Similarity

In [13]:
import numpy as np

def cosine_similarity (A , B) :

    A = np.array(A)
    B = np.array(B)

    maximum_similarity = 0
    target_class = -1

    norm_A = np.linalg.norm(A)

    for i in range(B.shape[0]) :

        norm_B = np.linalg.norm(B[i])
        similarity_score = (np.dot(A , B[i].T) / (norm_A * norm_B))[0]

        if similarity_score > maximum_similarity :

            maximum_similarity = similarity_score
            target_class = i

    return maximum_similarity , target_class

In [14]:
print(cosine_similarity(np.array([[1 , 2  ,3]]) , np.array([[1 , 2 , 3]])))

(1.0, 0)


## Define Model CLIP

In [15]:
from PIL import Image
import requests
from transformers import AutoProcessor , CLIPModel

In [16]:
model = CLIPModel.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K").to(device)
processor = AutoProcessor.from_pretrained("laion/CLIP-ViT-B-32-laion2B-s34B-b79K")

## Read queries folder image and store to numpy array

In [17]:
import cv2
import os
import numpy as np

In [18]:
refined_queries_path = '/content/drive/MyDrive/SuperAI_SS4/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/queries/refined_queries'
images = []

for file_name in os.listdir(refined_queries_path) :

    image = cv2.imread(f'{refined_queries_path}/{file_name}')
    image = refine_image(image)

    images.append(image)

inputs = processor(images = images , return_tensors = 'pt').to(device)
image_features = model.get_image_features(** inputs)

In [19]:
image_features = image_features.cpu().detach().numpy()

In [20]:
with open('/content/drive/MyDrive/SuperAI_SS4/SuperAI_SS4_Level_1/Hack_1_Image_Search/database/queries_matrix.npy' , 'wb') as file :

    np.save(file , image_features)


## Read test folder images to inference

In [21]:
from tqdm import tqdm
import pandas as pd

In [22]:
with open('/content/drive/MyDrive/SuperAI_SS4/SuperAI_SS4_Level_1/Hack_1_Image_Search/database/queries_matrix.npy' , 'rb') as file :

    queries_matrix = np.load(file)

In [23]:
queries_matrix.shape

(22, 1024)

In [24]:
submissions = {
    'img_file' : [] ,
    'class' : []
}

In [25]:
test_path = '/content/drive/MyDrive/SuperAI_SS4/SuperAI_SS4_Level_1/Hack_1_Image_Search/datasets/test/images'

for file_name in tqdm(os.listdir(test_path)) :

    image = cv2.imread(f'{test_path}/{file_name}')
    image = refine_image(image)

    inputs = processor(images = image , return_tensors = 'pt').to(device)
    image_features = model.get_image_features(** inputs)

    cosine_score , target_class = cosine_similarity(image_features.cpu().detach().numpy() , queries_matrix )

    if cosine_score < 0.75 :

        target_class = 22

    submissions['img_file'].append(file_name)
    submissions['class'].append(target_class)

100%|██████████| 1120/1120 [01:22<00:00, 13.64it/s]


In [26]:
submissions = pd.DataFrame.from_dict(submissions)
submissions_path = '/content/drive/MyDrive/SuperAI_SS4/SuperAI_SS4_Level_1/Hack_1_Image_Search/submissions/submissions.csv'

with open(submissions_path, 'wb') as csv_file:

    submissions.to_csv(path_or_buf = csv_file , index = False)

In [27]:
!kaggle competitions submit -c image-search -f '/content/drive/MyDrive/SuperAI_SS4/SuperAI_SS4_Level_1/Hack_1_Image_Search/submissions/submissions.csv' -m "submit"

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/dist-packages/kaggle/cli.py", line 54, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 806, in competition_submit_cli
    submit_result = self.competition_submit(file_name, message,
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 753, in competition_submit
    url_result = self.process_response(
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 3613, in process_response
    raise Exception(data['message'])
Exception: Unauthorized access
